# Celey: Asychronous backend tasks for Flask

This document will show the setup of Celery with the Flask web framework. At the end of this document you will know how to configure celery with rabbitmq as a messaging queue to perform asynchronous tasks from Flask, configure Supervisor to monitor Celery as a backend daemon, and troubleshoot bugs.

This document references example files that use pseudocode and will not function as is. I have also attached the actual project files to be used as reference

This document does not go into the full configuration of Flask but I will provided resources as necessary
Configuring Flask on EC2 Ubuntu 14.04 Server: https://www.datasciencebytes.com/bytes/2015/02/24/running-a-flask-app-on-aws-ec2/


## The 4 Components to the  Celery framework:
1. **Flask**: client application. This is where Celery is triggered to run an asychronous task. **simple_flask_example.py** shows a basic implementation of flask with the pseudocode to build a model
2. **Message Queue**: rabbitmq broker. This is the environment where celery "workers" perform the task. It can be configured such that rabbitmq runs the tasks on a separate server but in our example we run rabbitmq from the same ec2 environment
3. **Celery**: framework for asynchronous tasks. Celery need to be configuered to run with Flask. The file **celery_config.py** holds the configuration information
4. **Supervisor**: Process monitor/controller. This how we run Celery as a daemon backend process and monitior/troubleshoot Celery's performance or bugs.

## 1. Install packages

Rabbitmq

In [ ]:
sudo apt-get install rabbitmq-server

restart to make sure its running

In [ ]:
sudo service rabbitmq-server restart

Celery

In [ ]:
sudo pip install celery

Supervisor

In [ ]:
sudo apt-get install supervisor

In [ ]:
sudo service supervisor restart

## 2. Configure Flask to work with Celery

In your project directory copy the files **simple_flask_example.py**, this will be our sample flask application and **celery_config.py**, this contains the function **make_celery()** that **simple_flask_example.py** will call to configure Celery

## 3. How Celery Is Configured

In [ ]:
app = Flask(__name__)

# where you want the rabbitmq server running
app.config['CELERY_BROKER_URL'] = 'amqp://13.56.177.159//' 

# which server you want results to go to this can be configure with a datbase server
app.config['CELERY_BACKEND'] = 'amqp://13.56.177.159//'  

celery = make_celery(app)

## 4. How to make celery run an async task

In [ ]:
@celery.task(name = 'simple_flask_example.buildmodel_backend')
def buildmodel(filename):
    
    ### PROCESS TO BUILD MODEL
    
    return model


1. Define a celery task with the '@' decorator and give it an explict **name in the form 'name_of_flask_application.name_of_function**
2. Define a function to build a model. You can see how this is actually implement in the attached travelapi folder in **travelapi.py**

In [ ]:
@app.route('/buildmodelbackend/<filename>')
def buildmodelbackend(filename):
    buildmodel.delay(filename)
    return '202: Accepted'

3. Define an app route where we can call the flask application. In this example if we want to build a model wh can do so by going to **13.56.177.159/buildmodelbackend/12345**

4. Define a function at that app route (def buildmodelbackend(filename)) We tell flask call the celery function buildmodel by using the .delay method and passing it the argument (buildmodel.delay(filename)) This will start celery and then generate a 202 Response Code

## 5. Configuring Supervisord to monitor Celery as a backend process

New programs are given to Supervisor through configuration files, which inform it of the executable to run, any environmental variables, and how output should be handled.

All programs run under Supervisor must be run in a non-daemonising mode (sometimes also called 'foreground mode').

The program configuration files for Supervisor programs are found in the **/etc/supervisor/supervisord.conf**  directory, normally with one program per file and a .conf extension. Each program you supervisor to monitor need a configuration file saved at **/etc/supervisor/conf.d/{program_name}.conf**. In our case **/etc/supervisor/conf.d/celeryd.conf** for our celery program
 

1. Configure /etc/supervisor/supervisord.conf

In [ ]:
sudo nano /etc/supervisor/supervisord.conf

Make sure these are the same

2. Configure /etc/supervisor/conf.d/celeryd.conf

In [ ]:
sudo nano /etc/supervisor/conf.d/celeryd.conf

Make sure these are the same

After config files are saved run:

In [ ]:
sudo supervisorctl reread
sudo supervisorctl update
sudo supervisor reload

You can check the status of supervisor programs using:

In [ ]:
sudo supervisor status

This will tell you which supervisor programs are running or if there were any errors

More Resources for Supervisord: 
1. Config files:
https://github.com/celery/celery/tree/master/extra/supervisord
2. Config info:
http://supervisord.org/configuration.html
3. Supervisor on Ubuntu:
https://www.digitalocean.com/community/tutorials/how-to-install-and-manage-supervisor-on-ubuntu-and-debian-vps

## 6. Testing the whole process

Test whether Flask is correctly running celery in the backend by calling the endpoint at the defined app route:
13.56.177.159/buildmodelbackend/12345

1. Run the endpoint
2. Check the celery log configured in step 5 for the status of the celery work or if any errors occured

In [ ]:
cat /var/log/celery/worker.log